<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/pandas-profiling/colabs/Pandas_profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wnioski z analizy za pomocą pandas-profiling

## Uwaga

Analiza została stworzona na podstawie próbki o rozmiarze $1$ miliona obserwacji ze zbioru danych o aktorach. Dodatkowo, na cele tej analizy, ze zbioru zostały usunięte duplikaty aktorów (dla 1 aktora wystepuje tylko 1 wiersz w zbiorze).

## Kolumny o ziarnistej strukturze

Spośród $12$ zmiennych (w tym id aktora), aż $7$ przyjmuje bardzo dużo unikalnych wartości. Te zmienne to:
1. nconst (1000000 unikalnych wartości)
1. primaryName (930100 unikalnych wartości)
1. knownForTitles (797151 unikalnych wartości)
1. tconst (685420 unikalnych wartości)
1. originalTitle (549333 unikalnych wartości)
1. characters (342465 unikalnych wartości)

Kolumna nconst jest kluczowa, ponieważ jest ona identyfikatorem aktora. Pozostałe kolumny, bez żadnej dodatkowej ich obróbki, nie wnoszą większej wartości do analizy zbioru pod kątem podobieństwa aktorów. Tutaj zaznaczyć warto że kolumna knownForTitles zawiera wektory składające się z wielu filmów, dlatego znalezienie przynajmniej dwóch aktorów, ktorzy zasłynęli z pewnego filmu może być większa niż wskazuje na to obecna analiza.<br/>
Do tego dochodzi równiez kolumna genres, która ma 1871 unikalnych wartości, co jest znacznie mniejszą ilością niż w przypadku poprzednio wspomnianych kolumn, dlatego może być bardziej przydatna dla projektu.



## Ilośc brakujących danych

W całym zbiorze brakuje zaledwie mniej niż $0.1\%$ komórek, jednak prawie całe braki są skoncentrowane w trzech kolumnach:
1. characters
2. deathYear
3. birthYear


## Problematyczne kolumny


### birthYear i deathYear
Dla prawie $88\%$ aktorów nie ma podanej daty narodzin, a dla $96\%$ nie znamy day śmierci, a obie te kolumny są ze sobą mocno skorelowane. Uzupełnienie tych braków może przynieść zyski, dlatego powinniśmy znaleźć źródło dla tych danych.<br/>
Przy samej analizie data śmierci może być szczególnie problematyczna, gdyż informacji tej może brakować, albo aktor wciąż żyje.<br/>
Należy zwrócić uwagę, że minimalna wartość przyjmowana przez birthYear to $13$, a deathYear - $1569$. Może być to kwestia pojawiania się w tych kolumnach danych odstających.

### characters

Dla $26\%$ aktorów brakuje informacji o tym jakie role grali (kolumna characters).

### category i isAdult

Kolumna category przyjmuje jedynie dwie wartości - actor albo actress - a ich podział jest zbalansowany.
Podobnie, kolumna isAdult przyjmuje wartości $0$ i $1$, gdzie tym razem wartość $0$ jest przyjmowana przez ponad $95\%$ próbki. Ponadto jest ona mocno skorelowana z kolumną titleType.

### titleType i genres

W kolumnach titleType oraz genres nie wyróżniają się znacząco żadne wartości, jednak ich wartości różnią się od wybranego wiersza dla aktora (jeden wiersz to informacja o aktorze w tylko jednym filmie) i wartoby znaleźć sposób na połączenie tych danych w zakresie jednego aktora w listę, lub znaleźć inny sposób na wydobycie z tych kolumn użytecznych informacji.

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 200 kB 46.0 MB/s 
     |████████████████████████████████| 199 kB 62.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=4139ee055837c7dc6d78ef66f16abe320104d555dce9dbd80954ca5256842bcd
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-20 10:19:06--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").getOrCreate()  
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-20 10:19:37--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.94, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236619296 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.66M   115MB/s    in 2.0s    

2022-11-20 10:19:39 (115 MB/s) - ‘name.basics.tsv.gz’ saved [236619296/236619296]

--2022-11-20 10:19:39--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.94, 99.84.160.48, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287876715 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.54M  34.7MB

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [7]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [8]:
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [9]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (12089582, 5)
df_title_principals_selected dataframe size:  (20798916, 4)
df_title_basic_selected dataframe size:  (9381842, 5)
joined dataframe size:  (20795285, 8)
joined dataframe size:  (20795285, 12)


#Usuwamy duplikaty aktorów

In [10]:
clear_data = data.dropDuplicates(["nconst"])

In [11]:
print(f"Length of dataset after deleting duplicated actors: {clear_data.count()}")

Length of dataset after deleting duplicated actors: 2297092


# Analizujemy zbiór korzystając z pandas-profiling

## Zamieniamy dane na pandas dataframe

In [12]:
import pandas as pd
import numpy as np

In [13]:
sample = clear_data.rdd.takeSample(False, 1000000)
df_data = pd.DataFrame(sample, columns = clear_data.columns)

In [14]:
df_data = df_data.convert_dtypes()

In [15]:
df_data['deathYear'] = df_data['deathYear'].replace("\\N", np.NaN)
df_data['birthYear'] = df_data['birthYear'].replace("\\N", np.NaN)
df_data['deathYear'] = pd.to_numeric(df_data['deathYear'], downcast='float')
df_data['birthYear'] = pd.to_numeric(df_data['birthYear'], downcast='float')

In [16]:
df_data.head(10)

,tconst,nconst,primaryName,birthYear,deathYear,knownForTitles,category,characters,titleType,originalTitle,isAdult,genres
0,tt11792596,nm11354292,Riley John Costello,NaN,NaN,tt11792596,actor,"[""Cop 2""]",short,The Briefcase Incident,0,"Crime,Short"
1,tt0024729,nm0074010,Theodor Berge,1869.0,1946.0,"tt0035475,tt0024729,tt0022434",actor,"[""Breder""]",movie,Vi som går kjøkkenveien,0,"Comedy,Romance"
2,tt4278894,nm6647889,Khadija Parham,NaN,NaN,"tt4278894,tt3499632,tt7236092,tt6060448",actor,"[""Tranelle Swann (2015)""]",tvSeries,One Block,0,Comedy
3,tt0526572,nm0302931,Patrick Galligan,NaN,NaN,"tt1563137,tt0117716,tt0115326,tt0123816",actor,"[""Richard""]",tvEpisode,Artichoke Pie,0,Drama
4,tt0371507,nm1389917,Isabis,NaN,NaN,"tt0369207,tt0371505,tt0369206,tt0371506",actress,\N,video,Abbraxa: The Girls Must Pee,1,Adult
5,tt4157218,nm6876298,Claire Poulizac,NaN,NaN,tt4157218,actress,"[""Alison""]",short,Now Live,0,"Adventure,Drama,Mystery"
6,tt6296800,nm8621586,Kelly Richardson,NaN,NaN,tt6296800,actor,"[""Lisa""]",short,Unfunny,0,"Comedy,Short"
7,tt7061004,nm9052617,Eldren Keys,NaN,NaN,tt6993446,actor,"[""Reginald""]",tvEpisode,True Talents,0,Comedy
8,tt0095795,nm0379241,Alvis Hermanis,1965.0,NaN,"tt13853748,tt0301307,tt0311425,tt12907684",actor,"[""Svyatoslav Navarzin""]",movie,"Osen, Chertanovo...",0,"Crime,Drama,Romance"
9,tt12937560,nm11832535,Bryan Gage,NaN,NaN,\N,actor,"[""'guitar player'""]",video,Perfect,0,"Music,Short"


In [17]:
df_data.dtypes

tconst             string
nconst             string
primaryName        string
birthYear         float32
deathYear         float32
knownForTitles     string
category           string
characters         string
titleType          string
originalTitle      string
isAdult            string
genres             string
dtype: object

## Ogólne spojrzenie na informacje o zbiorze

In [18]:
!pip install pandas-profiling==3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 315 kB 4.6 MB/s 
     |████████████████████████████████| 690 kB 27.4 MB/s 
     |████████████████████████████████| 102 kB 51.8 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 9.9 MB 45.1 MB/s 
     |████████████████████████████████| 4.7 MB 37.9 MB/s 
     |████████████████████████████████| 296 kB 53.9 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=f60fbc8793a699eadebd5bb93f46160ea0abc5446991334921bf0523ae358e4d
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
Successfully built htmlmin
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2
  Attempting uninstall: requests
   

In [19]:
df_data.describe(include='all')

,tconst,nconst,primaryName,birthYear,deathYear,knownForTitles,category,characters,titleType,originalTitle,isAdult,genres
count,1000000,1000000,1000000,121546.000000,38297.000000,1000000,1000000,1000000,999693,999693,999693,999693
unique,685350,1000000,930781,NaN,NaN,797175,2,342929,10,549246,2,1859
top,tt2514504,nm11354292,Alex,NaN,NaN,\N,actor,\N,short,Episode #1.1,0,Drama
freq,9,1,63,NaN,NaN,92480,622013,258243,385168,3373,954336,100013
mean,NaN,NaN,NaN,1955.422852,1991.734863,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,31.822948,25.365746,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,13.000000,1569.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,1936.000000,1976.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,1963.000000,1998.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,1980.000000,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tworzymy raport o zbiorze danych

In [20]:
from pandas_profiling import ProfileReport

In [21]:
report = ProfileReport(df_data)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Zapisujemy raport do pliku html

In [22]:
report.to_file("Analysis.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]